In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
import re, math
import matplotlib.pyplot as plt
import seaborn as sns
import json
from IPython import display
from IPython.display import HTML


In [ ]:
#TODO...
def export_results(list_results_aggregated, results_astar, px = 2):
    columns_export = ['index','obj_mean', 'ttot_mean']
    raw_data = []
    for lra in list_results_aggregated:
        rawi = pd.read_csv(lra, sep="\s+")
        rawRestrict = rawi[columns_export]
        raw_data.append(rawRestrict)
    
    rawi = pd.read_csv(results_astar, sep="\s+")
    rawRestrict = rawi[['index','obj_mean', 'runs', 'ttot_mean']]
    raw_data.append(rawRestrict)
        
    raw_overall = raw_data[0]
    for i in range(len(raw_data)):
        if i > 0:
            raw_overall = pd.merge(raw_overall, raw_data[ i ], on = ['index'])
    
    # display polishing:
    # new data frame with split value columns 
    new = raw_overall["index"].str.split("_", n = 3, expand = True)
    #print(new)
    raw_overall['m'] = (new[ 0 ])
    raw_overall['n'] = (new[ 1 ])
    raw_overall['Sigma'] = (new[ 2 ])
    raw_overall['p'] = (new[ 3 ])
    
    raw_overall['m'] = raw_overall.apply(lambda x: int(x.m), axis = 1 )
    raw_overall['n'] = raw_overall.apply(lambda x: int(x.n), axis = 1 )
    raw_overall['Sigma'] = raw_overall.apply(lambda x: int(x.Sigma), axis = 1 )
    raw_overall['p'] = raw_overall.apply(lambda x: int(x.p), axis = 1 )
    raw_index = raw_overall[['Sigma', 'm', 'n', 'p' ]];
    
    # restrict by p-value
    raw_index = raw_index[ (raw_index['p']) == float(px) ]
    raw_overall = raw_overall[ raw_overall['p'] == px ]
    
    raw_overall = raw_overall.drop([ 'index', 'm', 'n', 'Sigma', 'p' ], axis = 1)
    raw_overall = pd.concat([ raw_index, raw_overall ], axis=1)
    raw_overall = raw_overall.sort_values(by= ['Sigma', 'm', 'n', 'p'], axis = 0)
    raw_overall = raw_overall.round(1)
    raw_overall.columns = range(0,len(raw_overall.columns))
    #add column max (appent to the end of table):
    raw_overall['max_value'] = raw_overall[[4, 6, 8, 10,  12, 14 ]].max(axis=1)
    raw_overall.rename(columns={ 0:'sigma', 1:'m', 2:'n', 3: 'p', 4: 'A_obj', 
                                 5: 'A_time', 6: 'G_obj', 7: 'G_time', 8: 'AP_obj', 
                                 9: 'GP_time', 
                                 10: 'BS1_obj', 11: 'BS1_time', 
                                 12: 'BS2_obj', 13: 'BS2_time',
                                 14: 'BS3_obj', 15: 'BS3_time',
                                 16: 'AStar_obj', 17: 'AStar_runs', 18: 'AStar_time' }, inplace=True)
    #boldface the best results...
    
    raw_overall['A_obj'] = raw_overall.apply(lambda x:   '$^{*}$' + str(x.A_obj)   if x.AStar_runs == 10 and float(x.AStar_obj) == float(x.A_obj)   else x.A_obj, axis=1)
    raw_overall['G_obj'] = raw_overall.apply(lambda x:   '$^{*}$' + str(x.G_obj)   if x.AStar_runs == 10 and float(x.AStar_obj) == float(x.G_obj)   else x.G_obj, axis=1)
    raw_overall['AP_obj'] = raw_overall.apply(lambda x:  '$^{*}$' + str(x.AP_obj)  if x.AStar_runs == 10 and float(x.AStar_obj) == float(x.AP_obj)  else x.AP_obj, axis=1)
    raw_overall['BS1_obj'] = raw_overall.apply(lambda x: '$^{*}$' + str(x.BS1_obj) if x.AStar_runs == 10 and float(x.AStar_obj) == float(x.BS1_obj) else x.BS1_obj, axis=1)
    raw_overall['BS2_obj'] = raw_overall.apply(lambda x: '$^{*}$' + str(x.BS2_obj) if x.AStar_runs == 10 and float(x.AStar_obj) == float(x.BS2_obj) else x.BS2_obj, axis=1)
    raw_overall['BS3_obj'] = raw_overall.apply(lambda x: '$^{*}$' + str(x.BS3_obj) if x.AStar_runs == 10 and float(x.AStar_obj) == float(x.BS3_obj) else x.BS3_obj, axis=1)

    raw_overall['A_obj'] = raw_overall.apply(lambda x:   '\\textbf{' + str(x.A_obj) + '}'   if float(x.max_value) == float(str(x.A_obj).replace('$^{*}$', ''))   else x.A_obj, axis=1)
    raw_overall['G_obj'] = raw_overall.apply(lambda x:   '\\textbf{' + str(x.G_obj) + '}'   if float(x.max_value) == float(str(x.G_obj).replace('$^{*}$', ''))   else x.G_obj, axis=1)
    raw_overall['AP_obj'] = raw_overall.apply(lambda x:  '\\textbf{' + str(x.AP_obj) + '}'  if float(x.max_value) == float(str(x.AP_obj).replace('$^{*}$', ''))  else x.AP_obj, axis=1)
    raw_overall['BS1_obj'] = raw_overall.apply(lambda x: '\\textbf{' + str(x.BS1_obj) + '}' if float(x.max_value) == float(str(x.BS1_obj).replace('$^{*}$', '')) else x.BS1_obj, axis=1)
    raw_overall['BS2_obj'] = raw_overall.apply(lambda x: '\\textbf{' + str(x.BS2_obj) + '}' if float(x.max_value) == float(str(x.BS2_obj).replace('$^{*}$', '')) else x.BS2_obj, axis=1)
    raw_overall['BS3_obj'] = raw_overall.apply(lambda x: '\\textbf{' + str(x.BS3_obj) + '}' if float(x.max_value) == float(str(x.BS3_obj).replace('$^{*}$', '')) else x.BS3_obj, axis=1)

    raw_overall = raw_overall.drop([ 'max_value', 'p', 'AStar_obj' ], axis = 1)
    #print(raw_overall)
    raw_latex = raw_overall.to_latex(index = False, escape=False)
    print(raw_latex)

#example:
export_results(['/home1/e01129111/results/dec-10m/clcs-alg-0bs-2000guidance-0-k_filter-100.csv',
                '/home1/e01129111/results/dec-10m/clcs-alg-1bs-2000guidance-0-k_filter-100.csv',
                '/home1/e01129111/results/dec-10m/clcs-alg-2bs-2000guidance-0-k_filter-100.csv',
                '/home1/e01129111/results/dec-10m/clcs-alg-3bs-1000guidance-0-k_filter-100.csv',
                '/home1/e01129111/results/dec-10m/clcs-alg-3bs-1000guidance-1-k_filter-100.csv',
                '/home1/e01129111/results/dec-10m/clcs-alg-3bs-1000guidance-3-k_filter-100.csv'
               ], '/home1/e01129111/results/dec-10m/r7.csv', 50)

In [ ]:
def export_results_clcs2(list_results_aggregated , px = 2):
    columns_export = ['index','obj_mean', 'ttot_mean']
    raw_data = []
    for lra in list_results_aggregated:
        rawi = pd.read_csv(lra, sep="\s+")
        rawRestrict = rawi[columns_export]
        raw_data.append(rawRestrict)
        
    raw_overall = raw_data[0]
    for i in range(len(raw_data)):
        if i > 0:
            raw_overall = pd.merge(raw_overall, raw_data[ i ], on = ['index'])
    
    # display polishing:
    # new data frame with split value columns 
    new = raw_overall["index"].str.split("_", n = 3, expand = True)
    #print(new)
    raw_overall['m'] = (new[ 0 ])
    raw_overall['n'] = (new[ 1 ])
    raw_overall['Sigma'] = (new[ 2 ])
    raw_overall['p'] = (new[ 3 ])
    
    raw_overall['m'] = raw_overall.apply(lambda x: int(x.m), axis = 1 )
    raw_overall['n'] = raw_overall.apply(lambda x: int(x.n), axis = 1 )
    raw_overall['Sigma'] = raw_overall.apply(lambda x: int(x.Sigma), axis = 1 )
    raw_overall['p'] = raw_overall.apply(lambda x: int(x.p), axis = 1 )
    raw_index = raw_overall[['Sigma', 'm', 'n', 'p' ]];
    
    # restrict by p-value
    raw_index = raw_index[ (raw_index['p']) == float(px) ]
    raw_overall = raw_overall[ raw_overall['p'] == px ]
    
    raw_overall = raw_overall.drop([ 'index', 'm', 'n', 'Sigma', 'p' ], axis = 1)
    raw_overall = pd.concat([ raw_index, raw_overall ], axis=1)
    raw_overall = raw_overall.sort_values(by= ['Sigma', 'm', 'n', 'p'], axis = 0)
    raw_overall = raw_overall.round(1)
    raw_overall.columns = range(0,len(raw_overall.columns))
    #add column min_time (appent to the end of table):
    raw_overall['min_time'] = raw_overall[[5, 7, 9, 11 ]].min(axis=1)
    raw_overall.rename(columns={ 0:'sigma', 1:'m', 2:'n', 3: 'p', 
                                 4: 'Chin_obj', 5: 'Chin_time', 
                                 6: 'AE_obj', 7: 'AE_time', 
                                 8: 'IR_obj', 9: 'IR_time', 
                                 10: 'AStar_obj', 11: 'AStar_time'}, inplace=True)
    
    #boldface the best results...
    raw_overall['Chin_time'] = raw_overall.apply(lambda x:    '\\textbf{' + str(x.Chin_time) + '}'   if float(x.min_time) == float(x.Chin_time)   else x.Chin_time, axis=1)
    raw_overall['AE_time'] = raw_overall.apply(lambda x:    "\\textbf{" + str(x.AE_time) + "}"   if float(x.min_time) == float(x.AE_time)   else x.AE_time, axis=1)
    raw_overall['IR_time'] = raw_overall.apply(lambda x:   "\\textbf{" + str(x.IR_time) + "}" if float(x.min_time) == float(x.IR_time)  else x.IR_time, axis=1)
    raw_overall['AStar_time'] = raw_overall.apply(lambda x:  '\\textbf{' + str(x.AStar_time) + '}' if float(x.min_time) == float(x.AStar_time) else x.AStar_time, axis=1)

    pd.testing.assert_series_equal(raw_overall['Chin_obj'], raw_overall['AE_obj'], check_names=False)
    pd.testing.assert_series_equal(raw_overall['Chin_obj'], raw_overall['IR_obj'], check_names=False)
    pd.testing.assert_series_equal(raw_overall['Chin_obj'], raw_overall['AStar_obj'], check_names=False)

    raw_overall = raw_overall.drop([ 'AE_obj'], axis = 1)
    raw_overall = raw_overall.drop([ 'IR_obj' ], axis = 1)
    raw_overall = raw_overall.drop([ 'AStar_obj' ], axis = 1)

                
    raw_overall = raw_overall.drop([ 'min_time', 'p' ], axis = 1)
    #print(raw_overall)
    raw_latex = raw_overall.to_latex(index = False, escape=False)
    print(raw_latex)

#example:
export_results_clcs2([
                '/home1/e01129111/nov-21/runs/sum/clcs-alg-4bs-1000guidance-3-k_filter-10.csv',
                '/home1/e01129111/nov-21/runs/sum/clcs-alg-5bs-1000guidance-3-k_filter-10.csv',
                '/home1/e01129111/nov-21/runs/sum/clcs-alg-6bs-1000guidance-3-k_filter-10.csv',
                '/home1/e01129111/nov-21/runs/sum/clcs-alg-7bs-1000guidance-3-k_filter-10.csv'
               ], 50)

In [ ]:
def export_results_clcs_real(list_results_aggregated , px = 2):
    columns_export = ['index','obj_mean', 'ttot_mean']
    raw_data = []
    for lra in list_results_aggregated:
        rawi = pd.read_csv(lra, sep="\s+")
        rawRestrict = rawi[columns_export]
        raw_data.append(rawRestrict)
        
    raw_overall = raw_data[0]
    for i in range(len(raw_data)):
        if i > 0:
            raw_overall = pd.merge(raw_overall, raw_data[ i ], on = ['index'])
    
    # display polishing:
    # new data frame with split value columns 
    new = raw_overall["index"].str.split("_", n = 1, expand = True)
    raw_overall['ds'] = (new[ 0 ])
    raw_overall['P'] = (new[ 1 ])
    raw_index = raw_overall[['ds', 'P']];
    raw_overall = raw_overall.drop(['index', 'ds', 'P'], axis = 1)
    raw_overall = pd.concat([ raw_index, raw_overall ], axis=1)
    
    raw_overall = raw_overall.sort_values(by= ['ds'], axis = 0)
    raw_overall = raw_overall.round(2)
    raw_overall.columns = range(0,len(raw_overall.columns))
    #add column min_time (appent to the end of table):
    raw_overall['min_time'] = raw_overall[[3, 5, 7, 9, 11 ]].min(axis=1)
    #print(raw_overall)
    raw_overall.rename(columns={ 0:'ds', 1:'P', 
                                 2: 'Chin_obj', 3: 'Chin_time', 
                                 4: 'AE_obj', 5: 'AE_time', 
                                 6: 'IR_obj', 7: 'IR_time', 
                                 8: 'AStar_obj', 9: 'AStar_time',
                                 10: 'MIP_obj', 11: 'MIP_time'}, inplace=True)
    
    #boldface the best results...
    raw_overall['Chin_time'] = raw_overall.apply(lambda x:    '\\textbf{' + str(x.Chin_time) + '}'   if float(x.min_time) == float(x.Chin_time)   else x.Chin_time, axis=1)
    raw_overall['AE_time'] = raw_overall.apply(lambda x:    "\\textbf{" + str(x.AE_time) + "}"   if float(x.min_time) == float(x.AE_time)   else x.AE_time, axis=1)
    raw_overall['IR_time'] = raw_overall.apply(lambda x:   "\\textbf{" + str(x.IR_time) + "}" if float(x.min_time) == float(x.IR_time)  else x.IR_time, axis=1)
    raw_overall['AStar_time'] = raw_overall.apply(lambda x:  '\\textbf{' + str(x.AStar_time) + '}' if float(x.min_time) == float(x.AStar_time) else x.AStar_time, axis=1)
    raw_overall['MIP_time'] = raw_overall.apply(lambda x:  '\\textbf{' + str(x.MIP_time) + '}' if float(x.min_time) == float(x.MIP_time) else x.MIP_time, axis=1)

    pd.testing.assert_series_equal(raw_overall['Chin_obj'], raw_overall['AE_obj'], check_names=False)
    pd.testing.assert_series_equal(raw_overall['Chin_obj'], raw_overall['IR_obj'], check_names=False)
    pd.testing.assert_series_equal(raw_overall['Chin_obj'], raw_overall['AStar_obj'], check_names=False)
    #pd.testing.assert_series_equal(raw_overall['Chin_obj'], raw_overall['MIP_obj'], check_names=False)

    raw_overall = raw_overall.drop([ 'AE_obj'], axis = 1)
    raw_overall = raw_overall.drop([ 'IR_obj' ], axis = 1)
    raw_overall = raw_overall.drop([ 'AStar_obj' ], axis = 1)
    raw_overall = raw_overall.drop([ 'MIP_obj' ], axis = 1)

                
    raw_overall = raw_overall.drop([ 'min_time'], axis = 1)
    #print(raw_overall)
    raw_latex = raw_overall.to_latex(index = False, escape=False)
    print(raw_latex)

#example:
export_results_clcs_real([
                '/home1/e01129111/results/dec-10r/clcs-alg-4.csv',
                '/home1/e01129111/results/dec-10r/clcs-alg-5.csv',
                '/home1/e01129111/results/dec-10r/clcs-alg-6.csv',
                '/home1/e01129111/results/dec-10r/clcs-alg-7.csv',
                '/home1/e01129111/results/dec-10r/clcs-real-time-900.csv'
               ], 2)

In [ ]:
def numerical_comparison(path_acs, path_bs, ObjOrUB = 1):
    
    raw_acs=pd.read_csv(path_acs, sep="\s+")
    raw_bs =pd.read_csv(path_bs, sep="\s+")
    raw_important_acs = raw_acs[['index', 'obj_mean', 'ub_mean', 'tbest_mean', 'ttot_mean' ]]
    raw_important_bs  = raw_bs[['index', 'obj_mean', 'ub_mean', 'tbest_mean', 'ttot_mean' ]]
    raw_overall = pd.merge(raw_important_acs, raw_important_bs, on = ['index'])

    # make it in boldface if obj_x > obj_y  (the same for ub )
    column_comparedX = 'obj_mean_x'
    column_comparedY = 'obj_mean_y'
    if ObjOrUB == 0:
        column_comparedX =  'ub_mean_x'
        column_comparedY =  'ub_mean_y'
        
    if ObjOrUB == 1:
       raw_overall['obj_mean_x_1'] = raw_overall.apply(lambda x: "*" + str(x.obj_mean_x ) if float(x.obj_mean_x) >= float(x.obj_mean_y) else x.obj_mean_x, axis = 1 )
       raw_overall['obj_mean_y_1'] = raw_overall.apply(lambda x: "*" + str(x.obj_mean_y ) if float(x.obj_mean_y) >= float(x.obj_mean_x) else x.obj_mean_y, axis = 1 )    
    else: # if ub is compared:
       raw_overall['ub_mean_x_1'] = raw_overall.apply(lambda x: "*" + str(round(x.ub_mean_x, 2) ) if float(x.ub_mean_x) <= float(x.ub_mean_y) else round(x.ub_mean_x, 2), axis = 1 )
       raw_overall['ub_mean_y_1'] = raw_overall.apply(lambda x: "*" + str(round(x.ub_mean_y, 2) ) if float(x.ub_mean_y) <= float(x.ub_mean_x) else round(x.ub_mean_y, 2), axis = 1 )    
    
    raw_overall[ column_comparedX ] =  raw_overall[column_comparedX + '_1']
    raw_overall[ column_comparedY ] =  raw_overall[column_comparedY + '_1']
    raw_overall = raw_overall.drop([ column_comparedX + '_1', column_comparedY + '_1' ], axis = 1)
    #print
    print("X algorithm: A*+ACS; Y algorithm: A* + BS")
    style1 = raw_overall.style.applymap(color_red, subset=[ column_comparedX, column_comparedY ]) # HTML object...
    print(display.display(style1))